# Criptografía - Práctica 2: Secuencias pseudoaleatorias
Guillermo Chica Sabariego

### Ejercicio 1: Postulados de Golomb

Hacemos una función que aplique los tres postulados de Golomb para saber si una secuencia es pseudoaleatoria. Para ello, dividimos primero los tres postulados en tres funciones diferentes.

#### Primer postulado:

In [8]:
def primerpostulado(s):
    '''
    Algoritmo que aplica el primer postulado de Golomb
    
    Entrada:
        s: secuencia binaria a comprobar en forma de lista
    
    Salida:
        True si en s hay |numero de ceros - numero de unos| <=1
        
        False en caso contrario
    '''

    ceros = s.count(0)
    unos = s.count(1)
    
    if (abs(ceros - unos)<=1):
        return True
    else:
        return False

#### Segundo postulado

In [9]:
def segundopostulado(s):
    '''
    Algoritmo que aplica el segundo postulado de Golomb
    
    Entrada:
        s: secuencia binaria en forma de lista
    
    Salida:
        True si el número de rachas de 1 es igual al de 0, siempre que sea posible
        
        False: En caso contrario
    '''
    ##Hacemos tabla con numero de rachas en orden creciente (diccionario o lista)
    #cada elemento i del diccionario debe ser el doble que el i+1
    #dos ultimas posiciones: comprobar solo que es mayor o igual
    rachas={}
    for i in range(1,len(s)+1): #como mucho habra rachas de la longitud de s
        rachas[i]=0
    repeticiones=1
    #recorro la lista incrementando un contador que pongo a 0 cuando haya un cambio
    #si el primero y el ultimo son iguales, empiezo en la segunda posición
    #si no, empiezo en la primera
    if(s[0]!=s[len(s)-1]):#si el primero y el ultimo son distintos
        for i in range(len(s)):
            if(i!=len(s)-1): #si no estoy en la última posición
                if(s[i]==s[i+1]):
                    repeticiones=repeticiones+1
                if(s[i]!=s[i+1]):#cambio
                    rachas[repeticiones]=rachas[repeticiones]+1
                    repeticiones=1

            if(i==len(s)-1): #si estoy en la última posición: se que ultimo y primero son disintos
                
                rachas[repeticiones]=rachas[repeticiones]+1
    else:
        #si el primero y el ultimo son iguales
        for i in range(1,len(s)): #empiezo en la segunda posición
            if(i!=len(s)-1): #si no estoy en la última posición
                if(s[i]==s[i+1]):
                    repeticiones=repeticiones+1
                if(s[i]!=s[i+1]):#cambio
                    rachas[repeticiones]=rachas[repeticiones]+1
                    repeticiones=1
            if(i==len(s)-1):#estoy en la última posición-primero y ultimo iguales
                repeticiones=repeticiones+1
                rachas[repeticiones]=rachas[repeticiones]+1
    #print rachas
    #Quito las últimas rachas que valen 0
    j=len(s)
    while(rachas[j]==0):
        rachas.pop(j)
        j=j-1
    #print rachas
    
    for i in range(1,len(s)):
        if (i<len(rachas)-1):

            if (rachas[i]!=rachas[i+1]*2):
                return False
        else: #estoy en la penúltima posición

            if (rachas[i]<rachas[i+1]):
                return False
            else:
                return True
    return True

#### Tercer postulado

Definimos antes unas fuciones auxiliares:


In [10]:
def hamming(s,d):
    '''
    Algoritmo que calcula la distancia hamming de dos listas binarias
    
    Entrada:
        s y d: listas binarias
    
    Salida:
        distancia: distancia hamming
    '''
    #Calcula la distancia hamming de dos listas
    #s lista original, d lista desplazada
    distancia=0
    for i in range(len(s)):
        if (s[i]!=d[i]):
            distancia = distancia + 1
    
    return distancia

In [11]:
def desplazar(s):
    '''
    Función que desplaza un lista una posición a la derecha
    
    Entrada:
        s: lista a desplazar
        
    Salida:
        d: lista desplazada
    '''
    d=[] #nueva lista desplazada
    d.append(s[len(s)-1])
    for i in range(len(s)-1):
        d.append(s[i])
    
    return d

In [12]:
def tercerpostulado(s):
    '''
    Función que aplica el tercer postulado de golomb
    
    Entrada:
        s: lista binaria
    
    Salida:
        True si la distancia hamming se mantiene constante entre la secuencia
        original y todos los posibles desplazamientos de s
        
        False en caso contrario
    '''
    
    d = desplazar(s)      
    distancia = hamming(s,d)
    #Hay que desplazar la cadena len(s)-1 veces
    #como ya la hemos desplazado una vez, hay que llegar hasta len(s)-2
    #Range para una unidad antes de llegar a stop
    #Equivale a parar cuando d==s
    for i in range(1,len(s)-1): 
        d = desplazar(d)
        if(distancia!=hamming(s,d)):
            return False
    return True

#### Golomb

Juntamos los tres postulados en una función

In [13]:
def golomb(s):
    '''
    Función que aplica el postulado de golomb a una secuencia binaria
    
    Entrada:
        s: secuencia binaria en forma de lista
    
    Salida:
        True si cumple los tres postulados
        False en caso contrario
    '''
    

    if(primerpostulado(s)==False): #si postulado da false, devuelve false
        return False
    if(segundopostulado(s)==False):
        return False
    if(tercerpostulado(s)==False):
        return False
    #Todos los postulados han dado True->devuelve True
    return True

Hacemos algunas comprobaciones. 

In [9]:
s=[0,0,0,1,1,1,1,0,1,0,1,1,0,0,1]
print golomb(s)

True


In [10]:
s1=[0,1,1,1,0,1,1,1,0,1,0]
print golomb(s1)


False


### Ejercicio 2: LFSR

Implementación de registros lineales de desplazamiento con retroalimentación (LFSR). Definimos antes una función de producto escalar entre dos secuencias binarias:

In [14]:
def productoescalar(c,r):
    """"
    Producto escalar modulo 2 de dos listas de 0 y 1
    
    Entrada:
        c: lista de 0 y 1 ej [1,1,1]
        r: lista de 0 y 1 ej [1,1,1]
    
    Salida:
        El producto escalar en modulo 2
    """
    s = 0
    for i in range(len(c)):
        s = s + c[i]*r[i]
    
    return s%2

In [15]:
def lfsr(c,s,l):
    """
    Implementación de lfsr
    
    Entrada:
        c: lista coeficientes polinomio conexión en orden [cL,...,c1]
    
        s: lista de semilla en orden [s0,...sL-1] de la misma longitud que c
    
        l: longitud secuencia salida
    
    Salida:
        s: resultado de aplicar el registro de desplazamiento a s
    """
    
    #Si la longitud de salida es menor o igual que el tamaño de la semilla
    if (l <= len(s)):
        return s[:l]
    
    #copiamos s en r que será el registro
    r=[]
    for i in s:
        r.append(i)
    
    for i in range(l):
        prod = productoescalar(c,r)
        #añado el nuevo valor
        s.append(prod)
        #muevo el registro a la izquierda
        r.pop(0) #quitamos el primer elemento
        r.append(prod) #añadimos el nuevo valor
    
    return s

A continuación se ilustra con ejemplos la dependencia del periodo de la semilla en el caso de polinomios reducibles. Por ejemplo (D⁴+D²+1) es reducible y su periodo es menor que el periodo máximo 2<sup>L</sup>-1 y depende de la semilla

In [25]:
print lfsr([1,0,1,0],[1,1,1,0],20) #periodo 6

[1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]


Vemos a simple vista que tiene periodo 6. Si cambiamos la semilla, cambia el periodo porque el polinomio de conexión es reducible:

In [16]:
print lfsr([1,0,1,0],[0,1,1,0],20) #periodo 3

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]


Este tiene periodo 3, a pesar de tener el mismo polinomio de conexión.

En el caso de polinomios irreducibles, el periodo no depende de la semilla,pero es divisor del periodo máximo 2**L-1. Veámoslo con el polinomio irreducible (D⁴+D³+D²+D):

In [17]:
print lfsr([1,1,1,1],[1,1,1,0],20) #periodo 5

[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]


In [18]:
print lfsr([1,1,1,1],[0,1,1,0],20)

[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]


In [26]:
print lfsr([1,1,1,1],[0,0,1,0],20)

[0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]


In [21]:
print lfsr([1,1,1,1],[0,0,1,1],20)

[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]


Siempre da periodo 5.

Si el polinomio de conexión es primitivo, el periodo es siempre 2<sup>L</sup>-1, independientemente de la semilla. Además, cumple los postulados de Golomb.

In [23]:
g = lfsr([1,0,0,0,0,1,0,0,0],[1,0,1,1,0,1,0,0,0],600) #El polinomio de conexión es primitivo
print golomb(g[0:(2**9)-1]) #el periodo es 2**L - 1 (L=9)

True


Y no va a depender de la semilla. La podemos cambiar:

In [24]:
g = lfsr([1,0,0,0,0,1,0,0,0],[1,0,1,1,0,1,1,1,0],600) #El polinomio de conexión es primitivo
print golomb(g[0:(2**9)-1]) #el periodo es 2**L - 1 (L=9)

True


### Ejercicio 3: NLFSR

Ahora implementamos un registro no lineal de desplazamiento asociado a una función polinómica f.

Hace falta una función de exponenciación modular:

In [27]:
def expmod(b,e,m):
    """
    Calcula la potencia modular de un número
    
    Entrada:
        b: base entera
        
        e: exponente entero
       
        m: modulo entero
    
    Salida:
        prod: resultado de (b^e) mod n
    """
    #b = base
    #e = exponente
    #m = modulo
    s=e
    prod=1 #En prod voy almacenando el resultado de las potencias
    a=b
    
    while(s>0):
        #Caso 1: primer bit es 1
        if(s%2==1):
            #Cambio prod
            prod = (prod * a) % m
            #Cambio s
            s = (s-1)/2
        #Caso 2: primer bit es 0
        elif (s%2==0):
            #No cambio prod
            #Cambio s
            s = s / 2
        #Cambio a
        a = (a*a) % m
        if(a==1):
            return prod
    
    #Devuelvo el resultado:
    return prod

In [28]:
def nlfsr(f, s, k):
    """
    Función que aplica el registro no lineal de desplazamiento asociado a f
    
    Entrada:
        f: función polinómica como lista de exponentes de monomios [[],[],[]]
        
        s: semilla en forma de lista [1,1,1...] de la misma longitud que f
        
        k: longitud del registro generado
        
    Salida:
        x: secuencia de longitud k obtenida al aplicar a s el registro no lineal
        de desplazamiento con retroalimentación asociado a f
    """
    
    #Si la longitu de salida es menor o igual que el tamaño de la semilla
    if(k<=len(s)):
        return s[:k]
    
    x = list(s)
    total=0
    prod=1
    
    for i in range(k-len(s)): #repetimos k-len(s) veces, para tener una secuencia de tamaño k
        for l in range(len(f)):#repetimos tantas veces como listas haya en f
            for j in range(len(f[0])):#calculamos la potencia en cada indice de s y f
                prod = prod* expmod(s[j],f[l][j],2)
            
            total = (total + prod)%2
            prod = 1
        #Ya hemos hecho la suma para todas las lista dada una semilla
        #Cambiamos semilla y repetimos
        x.append(total)
        s.pop(0)
        s.append(total)
        total = 0
    
    return x

Vamos a aplicar el NLFSR para una función f = ((x^y)∨¬z) donde ¬z es z negado, el and ^ se traduce en producto y el or xvy en x+y + xy. El polinomio resultante en forma de lista es [[1,1,1,0],[0,0,0,0],[0,0,1,0],[0,0,0,1]] y se obtiene porque f = 2xy + xyz + 1 + z + t = xyz + 1 z + t ya que las operaciones son en módulo 2. Los coeficientes de las listas representan los exponentes de las incógnitas de cada monomio de f.

In [29]:
f = [[1,1,1,0],[0,0,0,0],[0,0,1,0],[0,0,0,1]]
s = [1,0,1,1] #semilla
print nlfsr(f,s,100)

[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


Vemos que el periodo es 3 (100) pero no empieza a aparecer hasta después de 1, 0, 1, 1, 1, 1, 0.

### Ejercicio 4: Generador de Geffe

El generador de Geffe toma como entrada 3 LFSR x1,x2,x3 y aplica la función f(x1,x2,x3) = x1x2+x2x3+x3 donde el + es la operación xor.

In [30]:
def geffe(lfsr1, lfsr2, lfsr3):
    """
    Implementación del generador de geffe
    
    Entrada:
        LFSR1,LFSR2,LFSR3: 3 lfsr
        
    Salida:
        s: igual a lfsr1lfsr2+lfsr2lfsr3+lfsr3

    """
    
    s=[]
    
    for i in range(len(lfsr1)):
        s.append((lfsr1[i]*lfsr2[i]+lfsr2[i]*lfsr3[i]+lfsr3[i])%2)
    # La complejidad lineal de los lfsr es L1,L2 y L3
    #Sus periodos p1 = (2^L1-1) p2 = (2^L2-1) p3 = (2^L3-1)
    #Si L1,L2 y L3 son primos entre si, el periodo de s es p1*p2*p3
    return s

Si los periodos de lfsr1, lfsr2 y lfsr3 son primos entre sí, o lo que es lo mismo, si sus complejidades lineales L1,L2 y L3 son primos entre sí, el periodo de la secuencia obtenida co Geffe es p1*p2*p3. Si no lo fueran, el periodo sería el mínimo común múltiplo de los tres periodos.

In [34]:
#Ejemplo donde el periodo de salida sea p1p2p3
#Necesitamos 3 LFSR cuya L sea primos entre sí, por ejemplo:
#L1=3, L2=5,L3=2

lfsr1=lfsr([1,0,1],[1,0,0],97)
lfsr2=lfsr([1,1,0,1,1],[1,0,0,1,1],95)
lfsr3=lfsr([1,1],[1,0],98)
k=geffe(lfsr1,lfsr2,lfsr3)

print k

[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]


El periodo sale demasiado grande para poder visualizarlo...

Vamos a usar Geffe para obtener una clave k, calculada en el paso anterior, con la que hacer un cifrado de flujo.

In [35]:
def cifrado(m,k):
    """
    Implementación de un cifrado de flujo
    
    Entrada:
        m: mensaje a cifrar
        k: clave de la misma longitud que m
    
    Salida:
        c = m+k  (suma componente a componente modulo 2)
    """
    c=[]
    for i in range(len(m)):
        c.append((m[i]+k[i])%2)
    
    return c


In [36]:
def descifrado(c,k):
    """
    Descifrado de un cifrado de flujo
    
    Entrada:
        c: mensaje cifrado
        k: clave
    
    Salida:
        m = c+k (suma componente a componente modulo 2), el mensaje en claro
    """
    d =[]
    
    for i in range(len(c)):
        d.append((c[i]+k[i])%2)
        
    return d

In [39]:
m=[1,0,1,1,0,1,1,0,1,0,0,1,0,0,1,1,0,1,0,1]
print "mensaje: ", m
c = cifrado(m,k)
print "cifrado: ", c
d= descifrado(c,k)
print "descifrado: ",d

mensaje:  [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1]
cifrado:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1]
descifrado:  [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1]


### Ejercicio 5: Berlekamp-Massey

Dada una sucesión de bits periodica, podemos calcular su complejidad lineal y el polinomio de conexión que la genera con el algoritmo de berlekamp-massey:

In [2]:
def berlekamp(s):
    """
    Implementación del algoritmo berlekamp massey
    
    Entrada:
        s: secuencia binaria de tamaño n, en forma de lista, ordenada [s0,...sn-1]
    
    Salida:
        L: complejidad lineal de s
        C: polinomio de conexión que genera s con un lfsr
    """
    L=0
    C=[1]
    m=-1
    B=[1]
    N=0
    
    #d=1 cuando me encuentro el primer elemento de s que sea 1
    
    while (s[N]==0):
        N = N + 1
    
    #Primer elemento distinto de 0 -> d=1
    d=1
    T=list(C)
    #Cambiamos C
    #bd = B(D)*D^N-m
    Nm = N-m
    bd = list(B)
    for i in range(Nm): 
        bd.insert(0,0)
    #C + bd
    if (len(C)<=len(bd)):
        suma = list(bd)
        for i in range(len(C)):
            suma[i] = (C[i]+bd[i])%2
        C = list(suma)
    else:
        suma = list(C)
        for i in range(len(bd)):
            suma[i] = (C[i]+bd[i])%2
        C = list(suma)
    
    if (L<=N/2):
        L = N +1-L
        m = N
        B = list(T)
    
    N = N+1

    while (N<len(s)):
        
        #Calculamos discrepancia
        #aux = sumatoria de C(i)*s(N-i)
        aux = 0
        for i in range(1,L+1): #sumatoria de i=1 hasta L
            aux = aux + C[i]*s[N-i]
        d = (s[N] + aux)%2
        
        if (d==1):
            T=list(C)
            #Cambiamos C
            #bd = B(D)*D^N-m
            Nm = N-m
            bd = list(B)
            for i in range(Nm): 
                bd.insert(0,0)
            #C + bd
            if (len(C)<=len(bd)):
                suma = list(bd)
                for i in range(len(C)):
                    suma[i] = (C[i]+bd[i])%2
                C = list(suma)
            else:
                suma = list(C)
                for i in range(len(bd)):
                    suma[i] = (C[i]+bd[i])%2
                C = list(suma)
            
            if (L<=N/2):
                L = N +1-L
                m = N
                B = list(T)
               
            
        N = N+1
    

    return [L,C]

In [44]:
s = [1,0,1,1,1,1,0,0,0,1,0,0,1,1]
x = berlekamp(s)
print 'L = ',x[0]
print 'C = ',x[1]

L =  4
C =  [1, 0, 0, 1, 1]


Comprobamos que el polinomio de conexión C que devuelve genera la secuencia s original al aplicarle un LFSR. Hay que tener en cuenta que la función berlekamp devuelve C ordenado de c0,...,cN, y en nuestra función lfsr, lo introducimos al reves, de cN hasta c1, sin contar el termino independiente 1, que es c0. Como semilla usamos la secuencia s cortada a la longitud de C.

In [49]:
print lfsr([1,1,0,0],[1,0,1,1],20)
print s

[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]


Por último, hacemos ejemplos con sumas y productos de secuencias para ver qué ocurre con la complejidad lineal. Si las secuencias se han obtenido por LFSR usando polinomios de conexión primitivos, la suma de ellas tendrá una complejidad lineal que es la suma de las complejidades lineales de los polinomios de conexión generadores. Con el producto pasa igual, la complejidad lineal del producto sera el producto de las complejidades lineales.

Ejemplo. Sean dos polinomios de conexión c1 = D<sup>2</sup> + D + 1 y c2 = D<sup>3</sup> + D + 1. Sus complejidades lineales son respectivamente L1 = 2 y L2 = 3. Consideramos las secuencias que generan por LFSR a partir de las semillas 11 y 111 respectivamente:

In [52]:
c1 = [1,1]
c2 = [1,0,1]
x1 = lfsr(c1,[1,1],40)
print 'x1 = ',x1
x2 = lfsr(c2,[1,1,1],40)
print 'x2 = ',x2

x1 =  [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
x2 =  [1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1]


La primera secuencia resultante x1 tiene periodo 3 y la segunda, periodo 7. Si ahora realizamos la suma de ambas, las complejidades lineales se suman y el periodo el el mcm de 3 y 7, es decir, 21.

In [58]:
#Sumamos x1 + x2
suma = list(x1)
for i in range(len(x1)):
    suma[i] = (x1[i]+x2[i])%2
print suma

[0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]


In [61]:
print 'L = ',berlekamp(suma)[0]
print 'C = ',berlekamp(suma)[1]

L =  5
C =  [1, 0, 0, 0, 1, 1]


Se observa que se obtiene una complejidad lineal L=5, que es suma de L1+L2=2+3. El polinomio de conexión obtenido es D<sup>5</sup> + D<sup>4</sup> + 1. Comprobamos que este polinomio genera la secuencia suma:

In [63]:
print lfsr([1,1,0,0,0],[0,0,1,1,0],20)

[0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]


Veamos qué ocurre con el producto:

In [56]:
prod = list(x1)
for i in range(len(x1)):
    prod[i] = (x1[i]*x2[i])%2
print prod

[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]


In [62]:
print 'L = ',berlekamp(prod)[0]
print 'C = ',berlekamp(prod)[1]

L =  6
C =  [1, 1, 1, 0, 1, 0, 1]


Ahora la complejidad lineal del producto es L=6 que es igual a L1·L2 = 2·3. Además se obtiene un polinomio de conexión D<sup>6</sup> + D<sup>4</sup> + D<sup>2</sup> + D + 1. Comprobamos que el polinomio de conexión genera la secuencia producto:

In [64]:
print lfsr([1,0,1,0,1,1],[1,1,0,0,1,0],20)

[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1]
